In [41]:
n_embed = 32
with open('data.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [3]:
print(f"length of the dataset characters {len(text)}")

length of the dataset characters 1115394


In [4]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
#Creating a map for each characters to it's index as integer map
chartint = {ch:i for i,ch in enumerate(chars)}
inttchar = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [chartint[c] for c in s]
decode = lambda l: ''.join([inttchar[i] for i in l])

In [7]:
print(decode([46, 43, 50, 50, 53]))
print(encode("hello"))

hello
[46, 43, 50, 50, 53]


In [8]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
#splitting into training and validation data
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
torch.manual_seed(1337)
context_len = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    #randomized chunck pick
    r_cpick = torch.randint(len(data) - context_len, (batch_size,))
    # x = input data y = expected output
    x = torch.stack([data[i:i+context_len] for i in r_cpick])
    y = torch.stack([data[i+1:i+context_len+1] for i in r_cpick])
    return x, y

xb,yb = get_batch('train')
print(f'train stacked up chuncks:\n {xb}')
print(f'target stacked up chuncks:\n {yb}')


train stacked up chuncks:
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target stacked up chuncks:
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [53]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramModel(nn.Module):
    def __init__(self):
        super().__init__()
        # creating a table where logits for each token corresponding to next one is read
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(context_len, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self,idx,targets=None):
        B,T = idx.shape
        token_emb = self.token_embedding_table(idx) # shape = (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T))
        x = token_emb + pos_emb
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self,idx,max_tokens):
        for _ in range(max_tokens):
            #get prediction through forward function
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            #apply softmax to get probability
            probs = F.softmax(logits, dim=-1) # (B, C)
            #sample from distribution
            idx_next = torch.multinomial(probs,num_samples=1) # (B, 1)
            #append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)
            
        return idx

m = BigramModel()
out, loss = m(xb,yb)
print(out.shape)
print(loss)        

idx = torch.zeros((1,1), dtype=torch.long)
# print(decode(m.generate(idx, max_tokens=100)[0].tolist()))

    

torch.Size([256, 65])
tensor(4.4802, grad_fn=<NllLossBackward0>)


In [17]:
#creating pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [18]:
batch_size = 32
for step in range(10000):
    xb,yb = get_batch('train')

    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5727508068084717


In [19]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_tokens=100)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y helti


In [20]:
##Getting into self-attention

In [21]:
torch.manual_seed(1337)
B,T,C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [22]:
# xbow => x bag of words => just means average
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev,0)
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [23]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [24]:
#version 2
ones_vec = torch.tril(torch.ones(T,T))
ones_vec = ones_vec / ones_vec.sum(1, keepdim=True)
xbow2 = ones_vec @ x
# might not show true due to tight floating point precision
torch.allclose(xbow,xbow2)


False

In [25]:
#version 3
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
xbow3[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])